# 2. ¿Cuán rápido puedes ordenar estos
productos?
Descripción
La seccion de ofertas https://www.mercadolibre.com.ar/ofertas#nav-header es una sección que agrupa las mejores ofertas de MELI, y a grandes rasgos es un listado de productos en oferta ordenados por un score de ML y distintas reglas de negocio.
Anexado al desafío, se encuentra un archivo “ordenamiento.csv” el cual tiene un listado de productos, con su score y categorías para este desafío.
Un sample de este csv se ve de la siguiente manera:

## Regras : 
1. El “domain_id” no se puede repetir en 4 posiciones consecutivas.
2. El “vertical” no se puede repetir en 1 posición consecutiva.
3. De existir el id 641416750 en el listado debe estar en la posición 3 siendo esta regla más fuerte que las demás.
4. De existir el id 22351223 en el listado debe estar en la posición 6 siendo esta regla más fuerte que las demás.
5. Las posiciones 9,10,11 deben tener sí o sí items de la categoría “HOME&DECO”
siendo esta regla más fuerte que la 1 y 2.
6. Cumpliendo estas condiciones, el ordenamiento debe respetar un ordenamiento de mayor score a menor.



## Tarefas :
 - Algoritmo de ordenação [X]
 - Testar opções de ordenação [X]
 - Testar regras e  Algoritmo [X]
    

In [130]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
dtitens = pd.read_csv('ordenamiento.csv')
dtitens = dtitens.sort_values(by=['score'],ascending=False)
dtitens.head(4)

,item_id,vertical,category,domain,score
2284,590602034,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9998
4793,609438042,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9996
2898,634352041,CE,MOBILE,MLC-CELLPHONES,0.9994
661,615879515,CE,MOBILE,MLC-CELLPHONES,0.9992


# A Estratégia de ordenação "Bubble sort"
 A ideia é a partir de uma lista ordenada por score, aplicar uma variação do "Bubble sort", varrendo a lista e quando achar um item fora regra,  "subir" outro elemento capaz de satisfazer os requisitos estabelecidos .
  Para as regras 3,4 e 5 foram feitos métodos específicos que são chamados nas suas posiçôes específicas. 
 


In [110]:
import time
class OrdenadorMLBubblesort:

    def __init__(self, lista):
        self.posdomain_id = 3
        self.posvertical = 1
        self.poscategory = 2

        self.lista = lista
        self.listasize = len(lista)
        self.numerotrocas =0
        self.exectime = 0
        self.idpos3 =641416750
        self.idpos6 = 22351223
        self.item3 = None
        self.item6 = None


    # get next vertical item <> lastvertical
    def getvertical(self, indice, vertical):
        while indice < self.listasize:
            if self.lista[indice][self.posvertical] != vertical:
                item = self.lista[indice]
                self.lista.remove(item)
                return  item
            indice += 1

        return None

    # get next domain_id item <> lastdomain_id
    def getdomain_id(self, indice, domain_id):
        while indice < self.listasize:
            if self.lista[indice][self.posdomain_id] != domain_id:
                item = self.lista[indice]
                self.lista.remove(item)
                return  item
            indice += 1

        return None
    
    # get next ITEM domain_id <> lastdomain_id vertical <> lastvertical
    def getdomain_id_vertical(self, indice, domain_id, vertical):
        while indice <  self.listasize:
            item  = self.lista[indice]
            if item[self.posdomain_id] != domain_id and item[self.posvertical] != vertical :
                self.lista.remove(item)
                return item
            indice +=1

        return None


    def troca_domainid_vertical(self, indice, domain_id, vertical ):
        itemn = self.getdomain_id_vertical(indice,domain_id,vertical )
        if itemn is not None:
            self.lista.insert(indice, itemn)
            return itemn[self.posdomain_id], itemn[self.posvertical]
        else:
            return '', ''

    

    def troca_domain_id(self, indice, item):
        itemn = self.getdomain_id(indice, item[self.posdomain_id])
        if itemn is not None:
            self.lista.insert(indice, itemn)
            return item[self.posdomain_id], item[self.posvertical]
        else:
            return ''

    def troca_vertical(self, indice, item):
        itemn = self.getvertical(indice, item[self.posvertical])
        if itemn is not None:
            self.lista.insert(indice, itemn)
            return itemn[self.posvertical]
        else:
            return ''

    # get itens da posicao 3 e 6     
    def getids(self):
        for item in self.lista:
            if item[0] == self.idpos3:
                self.item3 = item
                self.lista.remove(item)
            if item[0] == self.idpos6:
                self.item3 = item
                self.lista.remove(item)
                
    def getHOMEDECOs(self, indice):
        itens =[]
        while indice < self.listasize and  len(itens) < 3:
            if len(self.lista[indice]) < self.poscategory:
                print(indice)

            if self.lista[indice][self.poscategory] == 'HOME&DECOR':
                itens.append(self.lista[indice])
                self.lista.remove(self.lista[indice])
            indice += 1

        return itens

    #  coloca nas posiçoes 8-11 'HOME&DECOR'
    def trocaHOMEDECO(self):
        itens= self.getHOMEDECOs(8)
        for i, item in  enumerate(itens):
            self.lista.insert(8 + i , item)
            vertical, domain_id =  item[self.posvertical], item[self.posdomain_id]


        return  item,vertical, domain_id , 9 + i

    


    def ordena(self):
        start_time = time.time()
        self.getids()
        self.listasize = len(self.lista)
        lastdomain_id = '' # ultimo domain_id
        n_domain_id  =  0
        lastVertical = '' # ultimo lastVertical
        self.numerotrocas =0
        
        i =0
        while ( i < self.listasize -1):
         

            if i == 2 and self.item3 is not None:  ## item pos 3 
                self.lista.insert(i, self.item3)
                lastVertical = self.item3[self.posvertical]
                lastdomain_id = self.item3[self.posdomain_id]
                i = 3

            elif i == 5 and self.item6 is not None: ## item pos 6 
                self.lista.insert(5, self.item6)
                lastVertical = self.item6[self.posvertical]
                lastdomain_id = self.item6[self.posdomain_id]

            elif i == 8: ## regra 5 : posição 9-11 <> 'HOME&DECOR 
                item,lastVertical, lastdomain_id, i=  self.trocaHOMEDECO()


            item = self.lista[i]
            # nao permite duplicar vertical
            if item[self.posvertical] == lastVertical:
                self.numerotrocas += 1
                if item[self.posdomain_id] == lastdomain_id and n_domain_id > 3:  # nao permite duplicardomain_id
                    lastdomain_id, lastVertical = self.troca_domainid_vertical(i, lastdomain_id, lastVertical)
                else:
                    lastVertical = self.troca_vertical(i, item)
                    if item[self.posdomain_id] == lastdomain_id:
                        n_domain_id +=1

            # nao permite “domain_id” 4 vezes
            elif item[self.posdomain_id] == lastdomain_id:
                if n_domain_id > 3:
                    self.numerotrocas += 1
                    lastdomain_id, lastVertical = self.troca_domainid_vertical(i, lastdomain_id, lastVertical)
                else:
                    n_domain_id += 1
                    lastVertical = item[self.posvertical]


            else:
                lastVertical = item[self.posvertical]
                lastdomain_id = item[self.posdomain_id]
                n_domain_id = 0
            i += 1

        self.exectime =(time.time() - start_time) * 1000


In [117]:
dtitens = dtitens.sort_values(by=['score'],ascending=False)
lista = dtitens.values.tolist()
ordenadorl = OrdenadorMLBubblesort(lista)
ordenadorl.ordena()

print("Para ordenar uma lista com {} itens, o algoritmo utilizou {} trocas de posiçao  e levou {:.1f} ms ".format(len(lista), ordenadorl.numerotrocas , ordenadorl.exectime ))


Para ordenar uma lista com 5000 itens, o algoritmo utilizou 1659 trocas de posiçao  e levou 97.6 ms 


In [119]:
df = pd.DataFrame (ordenadorl.lista, columns =['item_id', 'vertical', 'category', 'domain', 'score'])
df.to_csv('ordenamiento_result.csv')


## Validar as regras estabelecidas 
   

In [133]:
def valida_regras() :
    dtresult = pd.read_csv('ordenamiento_result.csv')
    print("Validando regra 1 : El “domain_id” no se puede repetir en 4 posiciones consecutivas. ")
    listadoman = list(dtresult['domain'])
    listaDuplicados =0
    for i in range(12, len(listadoman) - 4):
        if listadoman[i] == listadoman[i+1]\
                and listadoman[i] == listadoman[i+1] \
                  and listadoman[i] == listadoman[i+2] \
                    and listadoman[i] == listadoman[i+3] :
                        listaDuplicados +=1
    print(" ->Número de “domain_id”  repetidos 4X:" ,  listaDuplicados)

    print("Validando regra 2 : El “vertical” no se puede repetir  en 1 posición consecutiva.  ")
    listavertical = list(dtresult['vertical'])
    listaDuplicados = 0
    for i in range(12, len(listavertical)):
        if listavertical[i - 1] == listavertical[i]:
            listaDuplicados +=1
    print(" ->Número de “vertical”  consecutivos:" ,  listaDuplicados)

    if list(dtresult['item_id'])[2] == 641416750 :
        print("Validando regra 3 : item_id 641416750  na posição 3  ")
    else:
        print("Validando regra 3 : item_id 641416750 nao encontrado ")

    if list(dtresult['item_id'])[5] == 22351223 :
        print("Validando regra 4 : item_id 22351223  na posição 6  ")
    else:
        print("Validando regra 4 : item_id 22351223 nao encontrado ")

    listaCat = list(dtresult['category'])
    encontrou =0
    for elemento in  listaCat[8:11]:
        if elemento  != 'HOME&DECOR':
            encntrou +=1
    print("Validando regra 5 : Números elementos nas posições 9-11 <> 'HOME&DECOR ", encontrou)

valida_regras()    

Validando regra 1 : El “domain_id” no se puede repetir en 4 posiciones consecutivas. 
 ->Número de “domain_id”  repetidos 4X: 0
Validando regra 2 : El “vertical” no se puede repetir  en 1 posición consecutiva.  
 ->Número de “vertical”  consecutivos: 0
Validando regra 3 : item_id 641416750  na posição 3  
Validando regra 4 : item_id 22351223 nao encontrado 
Validando regra 5 : Números elementos nas posições 9-11 <> 'HOME&DECOR  0


# Conclusões 
1. Para ordenar a lista fornecidas com 5000 itens, o algoritmo, implementado pela Classe OrdenadorMLBubblesort, realizou 1659 trocas de posiçao na lista e levou **97.6 ms para execultar** 
2. O método valida_regras foi criado para validar o resultado do algoritimo, verificando se todas as regras foram obedecidas.
